# Password Cracking & Credential Attacks - Hands-On Lab

**Part of HackLearn Pro - Module #16**

## Learning Objectives

In this interactive lab, you will:

1. Master **hash identification** and understand cryptographic hash algorithms (MD5, SHA-1, bcrypt, Argon2)
2. Perform **password cracking** using dictionary, mask, rule-based, and hybrid attacks
3. Analyze **GPU acceleration** and understand the 25-billion-times speed difference between MD5 and bcrypt
4. Implement **secure password storage** using modern algorithms (bcrypt cost 12, Argon2id)
5. Build **multi-factor authentication** (TOTP) compatible with Google Authenticator
6. Detect and prevent **credential stuffing attacks** with rate limiting and behavioral analysis

**MITRE ATT&CK Coverage**: T1110.002 (Password Cracking), T1110.004 (Credential Stuffing)

**Legal Notice**: These labs are for AUTHORIZED educational purposes only. Password cracking without authorization is illegal under the Computer Fraud and Abuse Act (CFAA) and equivalent international laws. Only crack passwords from systems you own or have explicit written permission to test. Penalties include fines up to $250,000 and imprisonment up to 10 years.

## Setup

Install required packages for password cracking exercises:

In [ ]:
# Install required packages
!pip install bcrypt argon2-cffi pyotp qrcode requests

In [ ]:
import hashlib
import bcrypt
import secrets
import string
import re
import time
from datetime import datetime, timedelta
from collections import defaultdict
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError
import pyotp
import qrcode
from io import BytesIO

print("All required packages imported successfully!")
print(f"Python version: {import sys; sys.version}")
print("Ready to begin password cracking exercises.")

---

## Lab 1: Hash Identification and Algorithm Analysis

Learn to identify hash types and understand the security implications of different algorithms.

### Hash Type Identification

In [ ]:
# Example password hashes from various algorithms
example_hashes = {
    'md5': '5f4dcc3b5aa765d61d8327deb882cf99',  # password
    'sha1': '5baa61e4c9b93f3f0682250b6cf8331b7ee68fd8',  # password
    'sha256': '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8',  # password
    'ntlm': '8846F7EAEE8FB117AD06BDD830B7586C',  # password
    'bcrypt': '$2b$12$R9h/cIPz0gi.URNNX3kh2OPST9/PgBkqquzi.Ss7KIUgO2t0jWMUW',  # password
    'argon2': '$argon2id$v=19$m=47104,t=1,p=1$c29tZXNhbHQ$G3gzWm4h5B6F7H3J9K0L1M2N3O4P5Q6R',  # example
}

def identify_hash(hash_value):
    """
    Identify hash type based on length and format.
    Returns likely hash algorithm.
    """
    hash_value = hash_value.strip()

    # Check for bcrypt
    if hash_value.startswith('$2a$') or hash_value.startswith('$2b$') or hash_value.startswith('$2y$'):
        cost = hash_value.split('$')[2]
        return f"bcrypt (cost factor: {cost})"

    # Check for Argon2
    if hash_value.startswith('$argon2'):
        variant = hash_value.split('$')[1]
        return f"Argon2 (variant: {variant})"

    # Check for PBKDF2
    if hash_value.startswith('pbkdf2'):
        return "PBKDF2"

    # Check for scrypt
    if hash_value.startswith('$scrypt$'):
        return "scrypt"

    # Check by length (hex-encoded hashes)
    if len(hash_value) == 32:
        return "MD5 (likely) or NTLM"
    elif len(hash_value) == 40:
        return "SHA-1 (likely)"
    elif len(hash_value) == 64:
        return "SHA-256 (likely)"
    elif len(hash_value) == 128:
        return "SHA-512 (likely)"
    else:
        return "Unknown hash type"

# Test hash identification
print("Hash Type Identification\n" + "=" * 70)
for hash_type, hash_value in example_hashes.items():
    identified = identify_hash(hash_value)
    hash_display = hash_value[:40] + "..." if len(hash_value) > 40 else hash_value
    print(f"\n{hash_type.upper()}:")
    print(f"  Hash: {hash_display}")
    print(f"  Identified as: {identified}")

### Generate Hashes with Different Algorithms

In [ ]:
# Test password
test_password = "SecureP@ssw0rd!"

print(f"Test Password: {test_password}\n" + "=" * 70)

# MD5 (INSECURE)
md5_hash = hashlib.md5(test_password.encode()).hexdigest()
print(f"\nMD5 (INSECURE):")
print(f"  {md5_hash}")
print(f"  Length: {len(md5_hash)} characters")
print(f"  Speed: 10 billion hashes/second (GPU)")

# SHA-1 (INSECURE)
sha1_hash = hashlib.sha1(test_password.encode()).hexdigest()
print(f"\nSHA-1 (INSECURE):")
print(f"  {sha1_hash}")
print(f"  Length: {len(sha1_hash)} characters")
print(f"  Speed: 12 billion hashes/second (GPU)")

# SHA-256 (TOO FAST for passwords)
sha256_hash = hashlib.sha256(test_password.encode()).hexdigest()
print(f"\nSHA-256 (TOO FAST):")
print(f"  {sha256_hash}")
print(f"  Length: {len(sha256_hash)} characters")
print(f"  Speed: 3 billion hashes/second (GPU)")

# bcrypt (SECURE)
bcrypt_hash = bcrypt.hashpw(test_password.encode(), bcrypt.gensalt(rounds=12))
print(f"\nbcrypt (SECURE):")
print(f"  {bcrypt_hash.decode()}")
print(f"  Cost factor: 12")
print(f"  Speed: 8 hashes/second")
print(f"  GPU resistance: Excellent (memory-hard)")

# Argon2 (BEST-IN-CLASS)
ph = PasswordHasher()
argon2_hash = ph.hash(test_password)
print(f"\nArgon2id (BEST-IN-CLASS):")
print(f"  {argon2_hash}")
print(f"  Memory cost: 46 MB")
print(f"  Speed: 5 hashes/second")
print(f"  GPU resistance: Excellent (memory-hard + time-hard)")

print("\n" + "=" * 70)
print("\nKey Takeaway: bcrypt and Argon2 are 2.5 BILLION times slower than MD5.")
print("This makes brute-force attacks infeasible with modern password hashing.")

### Hash Verification

In [ ]:
# Verify MD5 hash (simple comparison)
def verify_md5(password, hash_value):
    return hashlib.md5(password.encode()).hexdigest() == hash_value

# Verify bcrypt hash (constant-time comparison)
def verify_bcrypt(password, hash_value):
    return bcrypt.checkpw(password.encode(), hash_value.encode() if isinstance(hash_value, str) else hash_value)

# Verify Argon2 hash
def verify_argon2(password, hash_value):
    try:
        ph.verify(hash_value, password)
        return True
    except VerifyMismatchError:
        return False

# Test verification
print("Hash Verification Tests\n" + "=" * 70)

correct_password = test_password
wrong_password = "WrongPassword123!"

print(f"\nMD5 Verification:")
print(f"  Correct password: {verify_md5(correct_password, md5_hash)}")
print(f"  Wrong password: {verify_md5(wrong_password, md5_hash)}")

print(f"\nbcrypt Verification:")
print(f"  Correct password: {verify_bcrypt(correct_password, bcrypt_hash)}")
print(f"  Wrong password: {verify_bcrypt(wrong_password, bcrypt_hash)}")

print(f"\nArgon2 Verification:")
print(f"  Correct password: {verify_argon2(correct_password, argon2_hash)}")
print(f"  Wrong password: {verify_argon2(wrong_password, argon2_hash)}")

print("\nNote: bcrypt and Argon2 include salts automatically, preventing rainbow table attacks.")

### Performance Benchmarking

In [ ]:
import time

def benchmark_hash_algorithm(algorithm, password, iterations=100):
    """
    Benchmark hash algorithm performance.
    """
    start_time = time.time()

    if algorithm == 'md5':
        for _ in range(iterations):
            hashlib.md5(password.encode()).hexdigest()
    elif algorithm == 'sha256':
        for _ in range(iterations):
            hashlib.sha256(password.encode()).hexdigest()
    elif algorithm == 'bcrypt':
        for _ in range(iterations):
            bcrypt.hashpw(password.encode(), bcrypt.gensalt(rounds=12))
    elif algorithm == 'argon2':
        for _ in range(iterations):
            ph.hash(password)

    elapsed = time.time() - start_time
    hashes_per_second = iterations / elapsed

    return elapsed, hashes_per_second

# Benchmark different algorithms
print("Password Hashing Performance Benchmark (Python CPU)\n" + "=" * 70)

test_pass = "password123"

algorithms = {
    'md5': 1000,
    'sha256': 1000,
    'bcrypt': 10,  # Fewer iterations because it's slow
    'argon2': 10
}

for algo, iters in algorithms.items():
    elapsed, hps = benchmark_hash_algorithm(algo, test_pass, iters)
    print(f"\n{algo.upper()}:")
    print(f"  Iterations: {iters}")
    print(f"  Time: {elapsed:.4f} seconds")
    print(f"  Speed: {hps:.2f} hashes/second (Python CPU)")

print("\n" + "=" * 70)
print("\nNOTE: GPU cracking is MUCH faster for MD5/SHA-256:")
print("  - MD5: 10 billion hashes/second (NVIDIA RTX 4090)")
print("  - bcrypt cost 12: 8 hashes/second (GPU-resistant)")
print("  - Difference: 1.25 BILLION times faster for MD5!")

---

## Lab 2: Dictionary & Rule-Based Attacks

Simulate password cracking using wordlists and rule-based mutations.

### Create Sample Wordlist

In [ ]:
# Common passwords from RockYou breach (top 20)
common_passwords = [
    "123456", "password", "12345678", "qwerty", "123456789",
    "12345", "1234", "111111", "1234567", "dragon",
    "123123", "baseball", "iloveyou", "trustno1", "1234567890",
    "sunshine", "master", "welcome", "shadow", "ashley"
]

# Write wordlist to file
with open('wordlist.txt', 'w') as f:
    for password in common_passwords:
        f.write(password + '\n')

print(f"Created wordlist with {len(common_passwords)} common passwords")
print(f"\nTop 10 passwords:")
for i, password in enumerate(common_passwords[:10], 1):
    print(f"  {i}. {password}")

print(f"\nStatistic: The top 20 passwords were used by 50%+ of users in RockYou breach.")

### Dictionary Attack Simulation

In [ ]:
def dictionary_attack(target_hash, hash_type='md5'):
    """
    Simulate dictionary attack against hash.
    """
    attempts = 0
    start_time = time.time()

    with open('wordlist.txt', 'r') as f:
        for password in f:
            password = password.strip()
            attempts += 1

            # Hash the password
            if hash_type == 'md5':
                candidate_hash = hashlib.md5(password.encode()).hexdigest()
            elif hash_type == 'sha256':
                candidate_hash = hashlib.sha256(password.encode()).hexdigest()
            else:
                continue

            # Check if match
            if candidate_hash == target_hash:
                elapsed = time.time() - start_time
                return password, attempts, elapsed

    return None, attempts, time.time() - start_time

# Test dictionary attack
target_password = "sunshine"  # One of our wordlist passwords
target_md5 = hashlib.md5(target_password.encode()).hexdigest()

print(f"Target Hash (MD5): {target_md5}")
print(f"\nAttempting dictionary attack...\n")

cracked, attempts, elapsed = dictionary_attack(target_md5, 'md5')

if cracked:
    print(f"SUCCESS!")
    print(f"  Password: {cracked}")
    print(f"  Attempts: {attempts}")
    print(f"  Time: {elapsed:.4f} seconds")
    print(f"  Speed: {attempts/elapsed:.2f} attempts/second")
else:
    print(f"FAILED - Password not in wordlist")
    print(f"  Attempts: {attempts}")
    print(f"  Time: {elapsed:.4f} seconds")

print(f"\nNote: With GPU acceleration (Hashcat), MD5 dictionary attack: 10 billion passwords/second!")

### Rule-Based Mutations

In [ ]:
def apply_rules(base_password):
    """
    Apply common mutation rules to password.
    Simulates Hashcat/John the Ripper rules.
    """
    mutations = [base_password]  # Original

    # Capitalize first letter
    mutations.append(base_password.capitalize())

    # All uppercase
    mutations.append(base_password.upper())

    # Append digits (2020-2024)
    for year in range(2020, 2025):
        mutations.append(base_password + str(year))
        mutations.append(base_password.capitalize() + str(year))

    # Append special characters
    for special in ['!', '@', '#', '$', '!@', '123']:
        mutations.append(base_password + special)
        mutations.append(base_password.capitalize() + special)

    # Leetspeak substitutions
    leetspeak = base_password
    leetspeak = leetspeak.replace('a', '@')
    leetspeak = leetspeak.replace('e', '3')
    leetspeak = leetspeak.replace('i', '1')
    leetspeak = leetspeak.replace('o', '0')
    leetspeak = leetspeak.replace('s', '$')
    mutations.append(leetspeak)
    mutations.append(leetspeak.capitalize())

    # Reverse
    mutations.append(base_password[::-1])

    # Duplicate
    mutations.append(base_password + base_password)

    return list(set(mutations))  # Remove duplicates

# Test rule-based mutations
base = "password"
mutated = apply_rules(base)

print(f"Base password: {base}")
print(f"\nGenerated {len(mutated)} mutations:")
print("=" * 70)

for i, mutation in enumerate(sorted(mutated)[:20], 1):  # Show first 20
    print(f"  {i}. {mutation}")

if len(mutated) > 20:
    print(f"  ... and {len(mutated) - 20} more")

print(f"\nRule-based attacks crack 60-70% of passwords in typical breaches!")

### Hybrid Attack Simulation

In [ ]:
def hybrid_attack(target_hash, hash_type='md5'):
    """
    Combine dictionary + rule-based mutations.
    """
    attempts = 0
    start_time = time.time()

    with open('wordlist.txt', 'r') as f:
        for base_password in f:
            base_password = base_password.strip()

            # Generate mutations
            mutations = apply_rules(base_password)

            for password in mutations:
                attempts += 1

                # Hash the password
                if hash_type == 'md5':
                    candidate_hash = hashlib.md5(password.encode()).hexdigest()
                elif hash_type == 'sha256':
                    candidate_hash = hashlib.sha256(password.encode()).hexdigest()
                else:
                    continue

                # Check if match
                if candidate_hash == target_hash:
                    elapsed = time.time() - start_time
                    return password, attempts, elapsed

    return None, attempts, time.time() - start_time

# Test with mutated password
target_password = "Welcome2024"
target_md5 = hashlib.md5(target_password.encode()).hexdigest()

print(f"Target: {target_password}")
print(f"Target Hash (MD5): {target_md5}")
print(f"\nAttempting hybrid attack (dictionary + rules)...\n")

cracked, attempts, elapsed = hybrid_attack(target_md5, 'md5')

if cracked:
    print(f"SUCCESS!")
    print(f"  Password: {cracked}")
    print(f"  Attempts: {attempts:,}")
    print(f"  Time: {elapsed:.4f} seconds")
    print(f"  Speed: {attempts/elapsed:.2f} attempts/second")
else:
    print(f"FAILED")
    print(f"  Attempts: {attempts:,}")
    print(f"  Time: {elapsed:.4f} seconds")

print(f"\nHybrid attacks are highly effective against password policies requiring:")
print(f"  - Capital letter + digits (Welcome2024)")
print(f"  - Word + special character (Password!)")
print(f"  - Common patterns users follow")

---

## Lab 3: Mask Attack Optimization

Learn pattern-based cracking to reduce keyspace and crack time.

### Mask Attack Syntax

In [ ]:
# Hashcat mask placeholders
mask_charset = {
    '?l': string.ascii_lowercase,  # a-z
    '?u': string.ascii_uppercase,  # A-Z
    '?d': string.digits,            # 0-9
    '?s': '!@#$%^&*()_+-=[]{}|;:,.<>?',  # special characters
}

print("Hashcat Mask Attack Syntax\n" + "=" * 70)
print("\nCharacter Sets:")
for placeholder, charset in mask_charset.items():
    print(f"  {placeholder} = {charset[:20]}..." if len(charset) > 20 else f"  {placeholder} = {charset}")

print("\nCommon Mask Patterns:")
patterns = [
    ("?u?l?l?l?l?d?d", "Uppercase + 4 lowercase + 2 digits", "Summer24, Winter99"),
    ("?l?l?l?l?l?l?s?d?d", "6 lowercase + special + 2 digits", "secret!23"),
    ("?u?l?l?l?l?l?l?l?s", "Uppercase + 7 lowercase + special", "Password!"),
    ("?d?d?d?d?d?d", "6 digits (PIN)", "123456"),
]

for mask, description, example in patterns:
    print(f"\n  Mask: {mask}")
    print(f"  Pattern: {description}")
    print(f"  Example: {example}")

### Calculate Keyspace Size

In [ ]:
def calculate_keyspace(mask):
    """
    Calculate total combinations for a mask pattern.
    """
    keyspace = 1

    # Parse mask
    i = 0
    while i < len(mask):
        if mask[i] == '?' and i + 1 < len(mask):
            placeholder = mask[i:i+2]
            if placeholder in mask_charset:
                keyspace *= len(mask_charset[placeholder])
            i += 2
        else:
            # Literal character
            keyspace *= 1
            i += 1

    return keyspace

def estimate_crack_time(keyspace, hashes_per_second):
    """
    Estimate time to crack based on keyspace and speed.
    """
    seconds = keyspace / hashes_per_second

    # Convert to human-readable
    if seconds < 60:
        return f"{seconds:.2f} seconds"
    elif seconds < 3600:
        return f"{seconds/60:.2f} minutes"
    elif seconds < 86400:
        return f"{seconds/3600:.2f} hours"
    elif seconds < 31536000:
        return f"{seconds/86400:.2f} days"
    else:
        return f"{seconds/31536000:.2f} years"

# Test keyspace calculation
print("Keyspace & Crack Time Analysis\n" + "=" * 70)

test_masks = [
    "?d?d?d?d",           # 4-digit PIN
    "?u?l?l?l?l?d?d",    # Capital + 4 lowercase + 2 digits
    "?l?l?l?l?l?l?l?l",  # 8 lowercase
    "?a?a?a?a?a?a?a?a",  # 8 any character (26+26+10+32=94 chars)
]

# GPU speeds (NVIDIA RTX 4090)
md5_speed = 10_000_000_000  # 10 GH/s
bcrypt_speed = 8             # 8 H/s at cost 12

for mask in test_masks:
    keyspace = calculate_keyspace(mask)
    md5_time = estimate_crack_time(keyspace, md5_speed)
    bcrypt_time = estimate_crack_time(keyspace, bcrypt_speed)

    print(f"\nMask: {mask}")
    print(f"  Keyspace: {keyspace:,} combinations")
    print(f"  MD5 (10 GH/s): {md5_time}")
    print(f"  bcrypt cost 12 (8 H/s): {bcrypt_time}")

### Optimized Mask Generation from Statistics

In [ ]:
def analyze_password_patterns(passwords):
    """
    Analyze password list to identify common patterns.
    Similar to PACK (Password Analysis & Cracking Kit).
    """
    patterns = defaultdict(int)

    for password in passwords:
        pattern = ""
        for char in password:
            if char.isupper():
                pattern += "?u"
            elif char.islower():
                pattern += "?l"
            elif char.isdigit():
                pattern += "?d"
            else:
                pattern += "?s"
        patterns[pattern] += 1

    return patterns

# Analyze common passwords
test_passwords = [
    "Password1", "Welcome2024", "Summer99", "Secret!23",
    "Admin@2024", "Winter2024", "Spring23!", "Test1234",
    "Hello2024", "Welcome123", "Password!", "Admin2024"
]

pattern_stats = analyze_password_patterns(test_passwords)

print("Password Pattern Analysis\n" + "=" * 70)
print(f"\nAnalyzed {len(test_passwords)} passwords\n")
print("Most Common Patterns:")

for pattern, count in sorted(pattern_stats.items(), key=lambda x: x[1], reverse=True)[:5]:
    percentage = (count / len(test_passwords)) * 100
    print(f"  {pattern}: {count} ({percentage:.1f}%)")

print("\nOptimized Mask Strategy:")
print("  1. Start with most common patterns (highest probability)")
print("  2. Use incremental mode for different lengths")
print("  3. Combine with hybrid attacks for maximum efficiency")
print("\nPACK (Password Analysis Kit) can reduce crack time by 60-80%!")

---

## Lab 4: GPU Benchmarking & Performance Analysis

Understand the massive performance difference between insecure and secure hashing.

### Algorithm Speed Comparison

In [ ]:
# GPU performance data (NVIDIA RTX 4090)
gpu_benchmarks = {
    'NTLM': {
        'speed': 200_000_000_000,  # 200 GH/s
        'description': 'Windows NTLM (MD4-based)',
        'security': 'EXTREMELY WEAK'
    },
    'MD5': {
        'speed': 10_000_000_000,   # 10 GH/s
        'description': 'MD5 hash',
        'security': 'BROKEN'
    },
    'SHA-256': {
        'speed': 3_000_000_000,    # 3 GH/s
        'description': 'SHA-256 hash',
        'security': 'TOO FAST'
    },
    'bcrypt (cost 5)': {
        'speed': 110_000,          # 110 KH/s
        'description': 'bcrypt with cost factor 5',
        'security': 'WEAK'
    },
    'bcrypt (cost 12)': {
        'speed': 8,                # 8 H/s
        'description': 'bcrypt with cost factor 12',
        'security': 'STRONG'
    },
    'Argon2': {
        'speed': 5,                # 5 H/s
        'description': 'Argon2id (OWASP recommended)',
        'security': 'BEST-IN-CLASS'
    },
}

print("GPU Performance Benchmarks (NVIDIA RTX 4090)\n" + "=" * 70)

for algo, data in gpu_benchmarks.items():
    speed = data['speed']
    if speed >= 1_000_000_000:
        speed_str = f"{speed/1_000_000_000:.1f} GH/s"
    elif speed >= 1_000_000:
        speed_str = f"{speed/1_000_000:.1f} MH/s"
    elif speed >= 1_000:
        speed_str = f"{speed/1_000:.1f} KH/s"
    else:
        speed_str = f"{speed} H/s"

    print(f"\n{algo}:")
    print(f"  Speed: {speed_str}")
    print(f"  Description: {data['description']}")
    print(f"  Security: {data['security']}")

### 8-Character Password Crack Time Comparison

In [ ]:
# 8-character alphanumeric password
# Charset: 26 lowercase + 26 uppercase + 10 digits = 62 characters
# Keyspace: 62^8 = 218,340,105,584,896 combinations

keyspace_8char = 62 ** 8

print(f"8-Character Alphanumeric Password\n" + "=" * 70)
print(f"\nCharset: a-z, A-Z, 0-9 (62 characters)")
print(f"Keyspace: {keyspace_8char:,} combinations")
print(f"\nCrack Time Analysis:\n")

for algo, data in gpu_benchmarks.items():
    speed = data['speed']
    crack_time = estimate_crack_time(keyspace_8char, speed)

    print(f"{algo}: {crack_time}")

print("\n" + "=" * 70)
print("\nKey Takeaway:")
print("  - MD5: 3 days (completely insecure)")
print("  - bcrypt cost 12: 8,900+ years (secure!)")
print("  - Difference: bcrypt is 1.2 MILLION times slower (good for defense)")

### Cloud GPU Rental Economics

In [ ]:
# Cloud GPU pricing
cloud_gpu_pricing = {
    'AWS EC2 P4d (8× NVIDIA A100)': {
        'cost_per_hour': 32.77,
        'gpus': 8,
        'md5_speed': 10_000_000_000 * 8,  # 80 GH/s total
    },
    'Google Cloud A100': {
        'cost_per_hour': 25.00,
        'gpus': 4,
        'md5_speed': 10_000_000_000 * 4,  # 40 GH/s total
    },
}

print("Cloud GPU Rental Economics for Password Cracking\n" + "=" * 70)

for provider, data in cloud_gpu_pricing.items():
    print(f"\n{provider}:")
    print(f"  Cost: ${data['cost_per_hour']:.2f}/hour")
    print(f"  GPUs: {data['gpus']}")
    print(f"  MD5 Speed: {data['md5_speed']/1_000_000_000:.0f} GH/s")

    # Calculate cost to crack 8-char alphanumeric (MD5)
    crack_hours = keyspace_8char / data['md5_speed'] / 3600
    crack_cost = crack_hours * data['cost_per_hour']

    print(f"  8-char alphanumeric (MD5): {crack_hours:.2f} hours = ${crack_cost:.2f}")

print("\n" + "=" * 70)
print("\nAttack Economics:")
print("  - $1,000 budget can rent 40+ GPU-hours of cracking power")
print("  - Attackers don't need to buy expensive hardware")
print("  - Temporary rental for specific breach makes detection harder")
print("\nDefense: Use bcrypt/Argon2 to make GPU cracking infeasible!")

---

## Lab 5: Custom Wordlist Generation

Create target-specific wordlists for higher crack rates.

### Simulate Web Scraping (CeWL Concept)

In [ ]:
# Simulate target company website content
company_text = """
Welcome to TechCorp Solutions. Founded in 2015, we provide innovative
technology solutions for enterprise clients. Our team of experts delivers
exceptional service quality. Contact us at support@techcorp.com for more
information. Our mission is excellence through innovation. Join our team
of professionals dedicated to customer success. Products include CloudSync,
DataVault, SecureNet, and EnterpriseHub. Located in Seattle, Washington.
CEO: Jennifer Mitchell. CTO: David Anderson. Visit our careers page.
"""

def generate_wordlist_from_text(text, min_length=6):
    """
    Extract potential passwords from text.
    Simulates CeWL (Custom Word List Generator).
    """
    # Extract words
    words = re.findall(r'\b[a-zA-Z]+\b', text)

    # Filter by minimum length
    words = [word for word in words if len(word) >= min_length]

    # Generate variations
    wordlist = set()

    for word in words:
        # Original
        wordlist.add(word.lower())
        wordlist.add(word.capitalize())
        wordlist.add(word.upper())

        # Add years
        for year in range(2015, 2025):
            wordlist.add(word.capitalize() + str(year))
            wordlist.add(word.lower() + str(year))

        # Add common endings
        for suffix in ['!', '123', '!@', '@123']:
            wordlist.add(word.capitalize() + suffix)

    # Extract numbers (years, codes)
    numbers = re.findall(r'\b\d{4}\b', text)
    for num in numbers:
        wordlist.add(num)

    # Extract email username parts
    emails = re.findall(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b', text)
    for email in emails:
        username = email.split('@')[0]
        wordlist.add(username)
        wordlist.add(username.capitalize())

    return sorted(wordlist)

# Generate custom wordlist
custom_wordlist = generate_wordlist_from_text(company_text)

print(f"Custom Wordlist Generator (CeWL Concept)\n" + "=" * 70)
print(f"\nGenerated {len(custom_wordlist)} password candidates\n")
print("Sample Passwords (first 30):")

for i, password in enumerate(custom_wordlist[:30], 1):
    print(f"  {i}. {password}")

if len(custom_wordlist) > 30:
    print(f"  ... and {len(custom_wordlist) - 30} more")

print("\n" + "=" * 70)
print("\nKey Insight:")
print("  - Target-specific wordlists have 5-10× higher success rate")
print("  - Employees often use company names, products, locations")
print("  - CeWL tool can spider websites automatically")
print("  - Defense: Train employees not to use company-related passwords!")

### Extract Metadata from Content

In [ ]:
def extract_password_candidates(text):
    """
    Extract specific password candidates from text.
    """
    candidates = {
        'names': re.findall(r'\b[A-Z][a-z]+ [A-Z][a-z]+\b', text),
        'emails': re.findall(r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b', text),
        'years': re.findall(r'\b(19|20)\d{2}\b', text),
        'locations': re.findall(r'\b[A-Z][a-z]+,\s*[A-Z][a-z]+\b', text),
        'products': ['CloudSync', 'DataVault', 'SecureNet', 'EnterpriseHub'],  # Manual extraction
        'company': ['TechCorp'],
    }

    return candidates

# Extract metadata
metadata = extract_password_candidates(company_text)

print("Password Candidate Extraction\n" + "=" * 70)

for category, items in metadata.items():
    if items:
        print(f"\n{category.upper()}:")
        for item in items:
            print(f"  - {item}")

print("\n" + "=" * 70)
print("\nLikely Password Patterns:")
print("  - Jennifer2015, Mitchell2024 (Name + year)")
print("  - TechCorp2024!, TechCorp123 (Company + suffix)")
print("  - CloudSync2024, DataVault! (Product + suffix)")
print("  - Seattle2024, Washington! (Location + suffix)")

---

## Lab 6: Secure Password Storage Implementation

Build production-ready password hashing with bcrypt and Argon2.

### bcrypt Implementation

In [ ]:
class PasswordHasher_bcrypt:
    """
    Production-ready password hashing with bcrypt.
    """

    def __init__(self, cost_factor=12):
        """
        Initialize with cost factor (4-31).
        Recommended: 12-14 (8-64 hashes/second).
        """
        self.cost_factor = cost_factor

    def hash_password(self, password: str) -> bytes:
        """
        Hash password using bcrypt.
        """
        salt = bcrypt.gensalt(rounds=self.cost_factor)
        hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
        return hashed

    def verify_password(self, password: str, hashed: bytes) -> bool:
        """
        Verify password against bcrypt hash.
        """
        return bcrypt.checkpw(password.encode('utf-8'), hashed)

# Test bcrypt implementation
print("bcrypt Password Hashing Implementation\n" + "=" * 70)

bcrypt_hasher = PasswordHasher_bcrypt(cost_factor=12)

test_passwords = [
    "SecureP@ssw0rd!",
    "MyC0mpl3x!Pass",
    "Tr0ng&Passw0rd2024"
]

for password in test_passwords:
    # Hash password
    start = time.time()
    hashed = bcrypt_hasher.hash_password(password)
    hash_time = time.time() - start

    # Verify correct password
    start = time.time()
    verify_correct = bcrypt_hasher.verify_password(password, hashed)
    verify_time = time.time() - start

    # Verify wrong password
    verify_wrong = bcrypt_hasher.verify_password("WrongPassword", hashed)

    print(f"\nPassword: {password}")
    print(f"  Hash: {hashed.decode()}")
    print(f"  Hash time: {hash_time:.4f}s")
    print(f"  Verify time: {verify_time:.4f}s")
    print(f"  Verify correct: {verify_correct}")
    print(f"  Verify wrong: {verify_wrong}")

print("\n" + "=" * 70)
print("\nbcrypt Benefits:")
print("  - Automatic salting (prevents rainbow tables)")
print("  - Adjustable cost factor (future-proof)")
print("  - GPU-resistant (memory-hard)")
print("  - Industry standard (Django, Ruby on Rails, PHP)")

### Argon2 Implementation (OWASP Recommended)

In [ ]:
class PasswordHasher_Argon2:
    """
    Production-ready password hashing with Argon2id.
    OWASP recommended as of 2023.
    """

    def __init__(self, time_cost=1, memory_cost=47104, parallelism=1):
        """
        Initialize Argon2id hasher.
        Defaults: time_cost=1, memory_cost=47104 (46 MB), parallelism=1
        """
        self.hasher = PasswordHasher(
            time_cost=time_cost,
            memory_cost=memory_cost,
            parallelism=parallelism,
            hash_len=32,
            salt_len=16,
            type='id'  # Argon2id (hybrid)
        )

    def hash_password(self, password: str) -> str:
        """
        Hash password using Argon2id.
        """
        return self.hasher.hash(password)

    def verify_password(self, password: str, hashed: str) -> bool:
        """
        Verify password against Argon2 hash.
        """
        try:
            self.hasher.verify(hashed, password)
            return True
        except VerifyMismatchError:
            return False

# Test Argon2 implementation
print("Argon2id Password Hashing Implementation\n" + "=" * 70)

argon2_hasher = PasswordHasher_Argon2()

for password in test_passwords:
    # Hash password
    start = time.time()
    hashed = argon2_hasher.hash_password(password)
    hash_time = time.time() - start

    # Verify correct password
    start = time.time()
    verify_correct = argon2_hasher.verify_password(password, hashed)
    verify_time = time.time() - start

    # Verify wrong password
    verify_wrong = argon2_hasher.verify_password("WrongPassword", hashed)

    print(f"\nPassword: {password}")
    print(f"  Hash: {hashed[:60]}..." if len(hashed) > 60 else f"  Hash: {hashed}")
    print(f"  Hash time: {hash_time:.4f}s")
    print(f"  Verify time: {verify_time:.4f}s")
    print(f"  Verify correct: {verify_correct}")
    print(f"  Verify wrong: {verify_wrong}")

print("\n" + "=" * 70)
print("\nArgon2 Benefits:")
print("  - Winner of Password Hashing Competition (2015)")
print("  - Resistant to GPU, ASIC, and side-channel attacks")
print("  - Configurable memory, time, and parallelism")
print("  - OWASP recommended (2023+)")
print("  - Best-in-class security")

### Password Strength Validation

In [ ]:
def validate_password_strength(password: str) -> tuple[bool, list[str]]:
    """
    Validate password against OWASP requirements.
    Returns (is_valid, list_of_issues).
    """
    issues = []

    # Length check (NIST recommends 12+ characters)
    if len(password) < 12:
        issues.append("Password must be at least 12 characters")

    # Character diversity
    if not re.search(r'[A-Z]', password):
        issues.append("Password must contain at least one uppercase letter")

    if not re.search(r'[a-z]', password):
        issues.append("Password must contain at least one lowercase letter")

    if not re.search(r'\d', password):
        issues.append("Password must contain at least one digit")

    if not re.search(r'[!@#$%^&*(),.?":{}|<>]', password):
        issues.append("Password must contain at least one special character")

    # Common password check
    common_passwords = ['password', '123456', 'qwerty', 'admin', 'letmein', 'welcome', 'monkey']
    if password.lower() in common_passwords:
        issues.append("Password is too common")

    # Sequential characters
    if re.search(r'(012|123|234|345|456|567|678|789|890|abc|bcd|cde|def)', password.lower()):
        issues.append("Password contains sequential characters")

    # Repeated characters
    if re.search(r'(.)\1{2,}', password):
        issues.append("Password contains repeated characters (3+ times)")

    is_valid = len(issues) == 0
    return is_valid, issues

# Test password validation
print("Password Strength Validation\n" + "=" * 70)

test_cases = [
    "weak",
    "Password123",
    "Str0ng!P@ssw0rd",
    "MyC0mpl3x&Passw0rd2024!",
    "aaa123456",
]

for password in test_cases:
    valid, issues = validate_password_strength(password)
    print(f"\nPassword: {password}")
    print(f"  Valid: {valid}")
    if issues:
        print(f"  Issues:")
        for issue in issues:
            print(f"    - {issue}")
    else:
        print(f"  No issues - password meets all requirements")

---

## Lab 7: Multi-Factor Authentication (TOTP)

Implement Time-based One-Time Passwords compatible with Google Authenticator.

### TOTP Setup and Generation

In [ ]:
class MFAManager:
    """
    Multi-Factor Authentication manager using TOTP.
    Compatible with Google Authenticator, Authy, etc.
    """

    def __init__(self, issuer="HackLearn Pro"):
        self.issuer = issuer

    def setup_mfa(self, username: str):
        """
        Generate MFA secret and QR code for user enrollment.
        Returns (secret, qr_code_bytes, uri).
        """
        # Generate random base32 secret
        secret = pyotp.random_base32()

        # Create TOTP object
        totp = pyotp.TOTP(secret)

        # Generate provisioning URI
        uri = totp.provisioning_uri(
            name=username,
            issuer_name=self.issuer
        )

        # Generate QR code
        qr = qrcode.QRCode(version=1, box_size=10, border=5)
        qr.add_data(uri)
        qr.make(fit=True)

        img = qr.make_image(fill_color="black", back_color="white")
        buffer = BytesIO()
        img.save(buffer, format='PNG')
        qr_bytes = buffer.getvalue()

        return secret, qr_bytes, uri

    def verify_mfa_code(self, secret: str, user_code: str) -> bool:
        """
        Verify user-provided TOTP code.
        valid_window=1 allows 30 seconds before/after for clock skew.
        """
        totp = pyotp.TOTP(secret)
        return totp.verify(user_code, valid_window=1)

    def get_current_code(self, secret: str) -> str:
        """
        Get current TOTP code (for testing/display).
        """
        totp = pyotp.TOTP(secret)
        return totp.now()

# Test MFA setup
print("Multi-Factor Authentication (TOTP) Setup\n" + "=" * 70)

mfa_manager = MFAManager(issuer="HackLearn Pro")

# Setup MFA for test user
username = "user@hacklearn.com"
secret, qr_code, uri = mfa_manager.setup_mfa(username)

print(f"\nUser: {username}")
print(f"Secret Key: {secret}")
print(f"  (Store securely - needed for all future verifications)")
print(f"\nProvisioning URI:")
print(f"  {uri}")
print(f"\nQR Code: Generated ({len(qr_code)} bytes)")
print(f"  User scans this with Google Authenticator app")

# Save QR code
with open('totp_qr.png', 'wb') as f:
    f.write(qr_code)
print(f"\nQR code saved to: totp_qr.png")

### TOTP Code Verification

In [ ]:
# Generate current code (simulates Google Authenticator app)
current_code = mfa_manager.get_current_code(secret)

print(f"TOTP Code Verification\n" + "=" * 70)
print(f"\nCurrent TOTP code (changes every 30 seconds): {current_code}")
print(f"  This is what Google Authenticator would show")

# Verify correct code
is_valid_correct = mfa_manager.verify_mfa_code(secret, current_code)
print(f"\nVerify current code: {is_valid_correct}")

# Verify wrong code
wrong_code = "000000"
is_valid_wrong = mfa_manager.verify_mfa_code(secret, wrong_code)
print(f"Verify wrong code ({wrong_code}): {is_valid_wrong}")

# Test time window
print(f"\nTime Window Test:")
print(f"  TOTP codes are valid for 30 seconds")
print(f"  valid_window=1 allows ±30 seconds (tolerates clock skew)")
print(f"  Total acceptance window: 90 seconds")

# Simulate code generation over time
import datetime
totp = pyotp.TOTP(secret)

print(f"\nCode generation timeline:")
for offset in [-60, -30, 0, 30, 60]:
    timestamp = datetime.datetime.now().timestamp() + offset
    code = totp.at(timestamp)
    print(f"  {offset:+3d}s: {code}")

### MFA Security Impact

In [ ]:
print("MFA Security Impact Analysis\n" + "=" * 70)

print("\nWithout MFA:")
print("  Scenario: Password breach via phishing")
print("  Attacker has: Password hash (or plaintext)")
print("  Result: Immediate account compromise")
print("  Success rate: 100%")

print("\nWith MFA (TOTP):")
print("  Scenario: Password breach via phishing")
print("  Attacker has: Password hash (or plaintext)")
print("  Attacker needs: Current TOTP code (changes every 30s)")
print("  Result: Account protected (password alone insufficient)")
print("  Success rate: Near 0% (unless sophisticated phishing)")

print("\nStatistics:")
print("  - Microsoft (2022): MFA blocks 99.9% of automated attacks")
print("  - Google: MFA adoption reduces account takeover by 100%")
print("  - Even weak password + MFA > strong password alone")

print("\nRecommendations:")
print("  1. Enable MFA on all critical accounts")
print("  2. Use authenticator apps (TOTP) over SMS (SIM swapping risk)")
print("  3. Store backup codes securely")
print("  4. Consider hardware keys (FIDO2/U2F) for highest security")

print("\n" + "=" * 70)
print("\nKey Takeaway: MFA is the single most effective defense against")
print("password-based attacks. Even if password is compromised, MFA blocks")
print("99.9% of automated credential stuffing and brute-force attacks.")

---

## Lab 8: Credential Stuffing Detection & Prevention

Build systems to detect and block credential stuffing attacks.

### Rate Limiting Implementation

In [ ]:
class RateLimiter:
    """
    IP-based rate limiting to prevent brute-force and credential stuffing.
    """

    def __init__(self, max_attempts=5, window_minutes=15):
        self.max_attempts = max_attempts
        self.window = timedelta(minutes=window_minutes)
        self.attempts = defaultdict(list)  # IP -> [timestamps]

    def is_allowed(self, ip_address: str) -> bool:
        """
        Check if IP is allowed to make login attempt.
        """
        now = datetime.now()

        # Clean old attempts outside window
        self.attempts[ip_address] = [
            ts for ts in self.attempts[ip_address]
            if now - ts < self.window
        ]

        # Check if over limit
        if len(self.attempts[ip_address]) >= self.max_attempts:
            return False

        # Record attempt
        self.attempts[ip_address].append(now)
        return True

    def get_remaining_attempts(self, ip_address: str) -> int:
        """
        Get remaining login attempts for IP.
        """
        now = datetime.now()
        self.attempts[ip_address] = [
            ts for ts in self.attempts[ip_address]
            if now - ts < self.window
        ]
        return max(0, self.max_attempts - len(self.attempts[ip_address]))

# Test rate limiter
print("Rate Limiting for Login Protection\n" + "=" * 70)

rate_limiter = RateLimiter(max_attempts=5, window_minutes=15)

test_ip = "192.168.1.100"

print(f"\nSimulating login attempts from {test_ip}:")
print(f"Limit: {rate_limiter.max_attempts} attempts per {rate_limiter.window.seconds//60} minutes\n")

for attempt in range(1, 8):
    allowed = rate_limiter.is_allowed(test_ip)
    remaining = rate_limiter.get_remaining_attempts(test_ip)

    status = "ALLOWED" if allowed else "BLOCKED"
    print(f"Attempt {attempt}: {status} (remaining: {remaining})")

print("\n" + "=" * 70)
print("\nRate Limiting Benefits:")
print("  - Prevents brute-force attacks")
print("  - Slows credential stuffing")
print("  - Protects against automated attacks")
print("  - Recommended: 5 attempts per 15 minutes per IP")

### Account Lockout Policy

In [ ]:
class AccountLockoutManager:
    """
    Account-based lockout after repeated failed login attempts.
    """

    def __init__(self, max_failures=5, lockout_minutes=30):
        self.max_failures = max_failures
        self.lockout_duration = timedelta(minutes=lockout_minutes)
        self.failed_attempts = defaultdict(int)  # username -> count
        self.lockout_until = {}  # username -> datetime

    def record_failed_login(self, username: str) -> None:
        """
        Record failed login attempt.
        """
        self.failed_attempts[username] += 1

        # Check if should lock account
        if self.failed_attempts[username] >= self.max_failures:
            self.lockout_until[username] = datetime.now() + self.lockout_duration
            print(f"  Account locked until {self.lockout_until[username].strftime('%H:%M:%S')}")

    def record_successful_login(self, username: str) -> None:
        """
        Reset failed attempts on successful login.
        """
        self.failed_attempts[username] = 0
        if username in self.lockout_until:
            del self.lockout_until[username]

    def is_locked(self, username: str) -> bool:
        """
        Check if account is currently locked.
        """
        if username not in self.lockout_until:
            return False

        # Check if lockout expired
        if datetime.now() > self.lockout_until[username]:
            del self.lockout_until[username]
            self.failed_attempts[username] = 0
            return False

        return True

# Test account lockout
print("Account Lockout Policy\n" + "=" * 70)

lockout_manager = AccountLockoutManager(max_failures=5, lockout_minutes=30)

test_user = "user@example.com"

print(f"\nSimulating failed login attempts for {test_user}:")
print(f"Policy: {lockout_manager.max_failures} failures = {lockout_manager.lockout_duration.seconds//60} min lockout\n")

for attempt in range(1, 7):
    if lockout_manager.is_locked(test_user):
        print(f"Attempt {attempt}: BLOCKED (account locked)")
    else:
        print(f"Attempt {attempt}: Failed login recorded")
        lockout_manager.record_failed_login(test_user)

print(f"\nAccount status: {'LOCKED' if lockout_manager.is_locked(test_user) else 'ACTIVE'}")
print(f"Failed attempts: {lockout_manager.failed_attempts[test_user]}")

print("\n" + "=" * 70)
print("\nBest Practices:")
print("  - Lockout after 5-10 failed attempts")
print("  - Lockout duration: 15-30 minutes")
print("  - Email notification to user on lockout")
print("  - Monitor for distributed attacks (many IPs, same account)")
print("  - Consider CAPTCHA after 3 failed attempts")

### Credential Stuffing Detection (Behavioral Analysis)

In [ ]:
class CredentialStuffingDetector:
    """
    Detect credential stuffing attacks via behavioral analysis.
    """

    def __init__(self):
        self.login_attempts = []  # (timestamp, username, ip, success)

    def record_login_attempt(self, username: str, ip: str, success: bool) -> None:
        """
        Record login attempt for analysis.
        """
        self.login_attempts.append({
            'timestamp': datetime.now(),
            'username': username,
            'ip': ip,
            'success': success
        })

    def detect_credential_stuffing(self, window_minutes=5) -> dict:
        """
        Analyze login patterns for credential stuffing indicators.
        """
        window = timedelta(minutes=window_minutes)
        now = datetime.now()

        # Filter recent attempts
        recent = [
            attempt for attempt in self.login_attempts
            if now - attempt['timestamp'] < window
        ]

        if not recent:
            return {'threat_level': 'LOW', 'indicators': []}

        indicators = []

        # Indicator 1: High volume of login attempts
        if len(recent) > 100:
            indicators.append(f"High volume: {len(recent)} attempts in {window_minutes} minutes")

        # Indicator 2: Many different usernames from single IP
        ip_to_users = defaultdict(set)
        for attempt in recent:
            ip_to_users[attempt['ip']].add(attempt['username'])

        for ip, users in ip_to_users.items():
            if len(users) > 10:
                indicators.append(f"IP {ip}: {len(users)} different usernames")

        # Indicator 3: Low success rate (typical for credential stuffing)
        success_count = sum(1 for a in recent if a['success'])
        success_rate = success_count / len(recent)
        if success_rate < 0.05 and len(recent) > 20:
            indicators.append(f"Low success rate: {success_rate*100:.1f}% (typical for stuffing)")

        # Indicator 4: Rapid sequential attempts
        if len(recent) >= 2:
            time_diffs = []
            for i in range(1, len(recent)):
                diff = (recent[i]['timestamp'] - recent[i-1]['timestamp']).total_seconds()
                time_diffs.append(diff)

            avg_diff = sum(time_diffs) / len(time_diffs)
            if avg_diff < 2:  # Less than 2 seconds between attempts
                indicators.append(f"Rapid attempts: avg {avg_diff:.2f}s between logins (automated)")

        # Determine threat level
        if len(indicators) >= 3:
            threat_level = 'HIGH'
        elif len(indicators) >= 2:
            threat_level = 'MEDIUM'
        elif len(indicators) >= 1:
            threat_level = 'ELEVATED'
        else:
            threat_level = 'LOW'

        return {
            'threat_level': threat_level,
            'indicators': indicators,
            'total_attempts': len(recent),
            'success_rate': f"{success_rate*100:.1f}%"
        }

# Simulate credential stuffing attack
print("Credential Stuffing Detection\n" + "=" * 70)

detector = CredentialStuffingDetector()

print("\nSimulating credential stuffing attack...\n")

# Simulate 50 login attempts from single IP with different usernames
attacker_ip = "203.0.113.42"
for i in range(50):
    username = f"user{i}@example.com"
    success = (i % 25 == 0)  # 2% success rate (typical for stuffing)
    detector.record_login_attempt(username, attacker_ip, success)

# Analyze
analysis = detector.detect_credential_stuffing(window_minutes=5)

print(f"Threat Level: {analysis['threat_level']}")
print(f"Total Attempts: {analysis['total_attempts']}")
print(f"Success Rate: {analysis['success_rate']}")
print(f"\nIndicators:")
for indicator in analysis['indicators']:
    print(f"  - {indicator}")

print("\n" + "=" * 70)
print("\nDefense Recommendations:")
print("  1. Enable rate limiting (5 attempts per 15 min per IP)")
print("  2. Require CAPTCHA after 3 failed attempts")
print("  3. Implement MFA (blocks 99.9% of stuffing attacks)")
print("  4. Monitor for distributed attacks (many IPs)")
print("  5. Check passwords against Have I Been Pwned database")
print("  6. Block known malicious IPs (use threat intelligence feeds)")

---

## Summary: Key Metrics & Takeaways

In [ ]:
import json

# Summary statistics
summary = {
    'hash_algorithms': {
        'insecure': ['MD5 (10 GH/s)', 'SHA-1 (12 GH/s)', 'NTLM (200 GH/s)'],
        'secure': ['bcrypt cost 12 (8 H/s)', 'Argon2 (5 H/s)', 'scrypt']
    },
    'attack_types': {
        'dictionary': 'Cracks 30-50% of passwords',
        'hybrid': 'Cracks 60-70% of passwords',
        'mask': 'Reduces keyspace by 90%+ for common patterns',
        'rainbow_table': '100% for unsalted hashes, 0% for salted'
    },
    'gpu_performance': {
        'md5_vs_bcrypt_ratio': '1,250,000,000:1',
        '8char_crack_time_md5': '3 days',
        '8char_crack_time_bcrypt': '8,900 years'
    },
    'real_world_breaches': {
        'RockYou (2009)': '32M plaintext passwords',
        'LinkedIn (2012)': '117M SHA-1 unsalted (90% cracked)',
        'Adobe (2013)': '153M 3DES-ECB (pattern analysis)',
        'Ashley Madison (2015)': '36M bcrypt (strong, resisted cracking)',
        'COMB (2021)': '3.2B plaintext (compilation)',
        'LastPass (2022)': 'PBKDF2 vaults (weak master passwords vulnerable)'
    },
    'defense_strategies': {
        'password_hashing': 'Use bcrypt cost 12 or Argon2id',
        'mfa': 'Blocks 99.9% of automated attacks',
        'rate_limiting': '5 attempts per 15 min per IP',
        'account_lockout': '5 failures = 30 min lockout',
        'password_policy': '12+ chars, uppercase, lowercase, digit, special'
    },
    'statistics': {
        'breaches_with_stolen_creds': '86% (Verizon DBIR 2024)',
        'mfa_effectiveness': '99.9% block rate (Microsoft 2022)',
        'credential_stuffing_attacks': '193 billion globally (Akamai 2023)',
        'yoy_increase': '24%',
        'have_i_been_pwned': '11.9B accounts, 613M passwords'
    },
    'mitre_attack': {
        'T1110.001': 'Password Guessing',
        'T1110.002': 'Password Cracking',
        'T1110.003': 'Password Spraying',
        'T1110.004': 'Credential Stuffing'
    }
}

print("PASSWORD CRACKING & CREDENTIAL ATTACKS - LAB SUMMARY")
print("=" * 70)
print("\nTechniques Mastered:")
print(json.dumps(summary, indent=2))

print("\n" + "=" * 70)
print("\nKey Takeaways:")

print("\n1. Hash Algorithm Security:")
print("   - MD5/SHA-1/NTLM: BROKEN - 10-200 billion hashes/second (GPU)")
print("   - bcrypt/Argon2: SECURE - 5-8 hashes/second (GPU-resistant)")
print("   - Use bcrypt cost 12+ or Argon2id for production systems")

print("\n2. Attack Effectiveness:")
print("   - Dictionary: 30-50% crack rate (rockyou.txt: 14M passwords)")
print("   - Hybrid: 60-70% crack rate (dictionary + rules)")
print("   - Mask: 90%+ keyspace reduction for common patterns")
print("   - Rainbow tables: Instant for unsalted, useless for salted")

print("\n3. GPU Acceleration Impact:")
print("   - 8-char alphanumeric password (218 trillion combinations)")
print("   - MD5: 3 days to crack with RTX 4090")
print("   - bcrypt cost 12: 8,900 years to crack (1.25 billion times slower)")
print("   - Cloud rental: $32/hour for 8× A100 GPUs (80 GH/s MD5)")

print("\n4. Real-World Breaches:")
print("   - RockYou (2009): 32M plaintext - spawned rockyou.txt wordlist")
print("   - LinkedIn (2012): 117M SHA-1 unsalted - 90% cracked in days")
print("   - Ashley Madison (2015): 36M bcrypt - strong hashing resisted cracking")
print("   - COMB (2021): 3.2B credentials - still used in stuffing attacks")

print("\n5. Defense Strategies:")
print("   - Password Hashing: bcrypt cost 12 or Argon2id (OWASP recommended)")
print("   - Multi-Factor Auth: Blocks 99.9% of automated attacks (Microsoft)")
print("   - Rate Limiting: 5 attempts per 15 min per IP")
print("   - Account Lockout: 5 failures = 30 min lockout")
print("   - Password Policy: 12+ chars, mixed case, digits, special")

print("\n6. Credential Stuffing:")
print("   - 193 billion attacks detected globally (2023)")
print("   - 24% year-over-year increase")
print("   - 0.1%-2% success rate per site (devastating at scale)")
print("   - MFA is most effective defense (99.9% block rate)")

print("\n" + "=" * 70)
print("\nReal-World Impact:")
print("  - Verizon DBIR 2024: 86% of breaches involve stolen credentials")
print("  - Average cost per breached record: $150")
print("  - Password '123456' used by 37 million accounts (HIBP)")
print("  - Password 'password' used by 9.5 million accounts (HIBP)")

print("\n" + "=" * 70)
print("\nEthical Reminder:")
print("  Password cracking without authorization is ILLEGAL.")
print("  CFAA (US), Computer Misuse Act (UK), and similar laws worldwide.")
print("  Penalties: Fines up to $250,000, imprisonment up to 10 years.")
print("\n  Use these skills for:")
print("    - Authorized penetration testing with written permission")
print("    - Your own systems and passwords")
print("    - Educational labs (HackTheBox, TryHackMe, etc.)")
print("    - Bug bounty programs with explicit scope")
print("    - Security research with proper disclosure")

print("\n" + "=" * 70)
print("\nNext Steps:")
print("  1. Practice on legal platforms (HackTheBox, TryHackMe, OSCP labs)")
print("  2. Set up your own lab with Kali Linux + Metasploitable")
print("  3. Learn Hashcat advanced features (mask files, rule generators)")
print("  4. Study password analysis (pipal, PACK)")
print("  5. Implement secure authentication in your own projects")
print("  6. Explore MITRE ATT&CK Credential Access tactics")

print("\n" + "=" * 70)
print("\nCongratulations! You've completed the Password Cracking lab.")
print("You've mastered hash algorithms, GPU acceleration, and modern authentication security.")
print("=" * 70)